Assignment 3 - Diachronic analysis of movie dialogues
===

*Due: January 1 2023*

In this assignment you will analyze how the expression of Hope in movie dialogues changed with time

You will follow these steps:

1. Find an appropriate control measure to contrast with Hope

2. Find psychometric tools or equivalent to generate initial bags of seeds, for both Hope and the control condition

3. Use WordNet tools to expand your bag of seeds and get synonyms and hyponyms, and start excluding words with unrelated meanings using the filters available.

4. Train a semantic vector space model using word2vec

5. Use your vector space model to find out the semantic clouds of each word in your bag of seeds, and select only the words with semantically meaningful clouds

6. Take the final Bag of Words, prune it further if necessary, and use the second script to calculate word frequencies (don't forget to upload the text files)

7. Calculate your psychological relevant ratio, build a dataframe and plot a time series using Seaborn

In [1]:
#
#
#